In [ ]:
import os, torch
from sklearn.model_selection import train_test_split
import pickle
import torch_geometric.transforms as T
import numpy as np
from torch_geometric.nn.models import Node2Vec
from torch_geometric.data import DataLoader
from torch_geometric.nn import MessagePassing
from torch_geometric.data import Data
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

epochs = int(os.getenv("EPOCHS", 1000))  # Default to 10 if not provided
learning_rate = float(os.getenv("LEARNING_RATE", 0.001))  # Default to 0.001
hidden_c = int(os.getenv("HIDDEN_C", 128))  # Default to 16
random_seed = int(os.getenv("RANDOM_SEED", 20))  # Default to 42
api_key = os.getenv("API_KEY", None)
graph_num = os.getenv("GRAPH_NUM", 17)
dropout_p = float(os.getenv("DROPOUT", 0.5))

# wandb.login()
# run = wandb.init(
#     project="graph-embedding",
#     config={
#         "epochs": epochs,
#         "learning_rate": learning_rate,
#         "hidden_c": hidden_c,
#         "random_seed": random_seed,
#         "num_layers": num_layers,
#         "dropout_p": dropout_p
#     }
# )

if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"Using CUDA device: {torch.cuda.get_device_name(0)}", flush = True)
else:
    device = torch.device('cpu')
    print("Using CPU", flush = True)

### load graph data

with open(f'../data/graphs/{graph_num}/linegraph_tg.pkl', 'rb') as f:
    data = pickle.load(f)

data.edge_index = data.edge_index.contiguous()
data.x = data.x.contiguous()
data.y = data.y.contiguous()

sc = StandardScaler()
data.x = torch.tensor(sc.fit_transform(data.x.cpu().numpy()))
